In [ ]:
%reset -f
import pandas as pd
import numpy as np
from wavhandler import *
from utils_train import *
import soundfile as sf
from keras.applications.densenet import DenseNet121
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

data1 = Dataset('Wingbeats')
data1.read(data='all', setting='read', labels='text', loadmat=False)

In [ ]:
seed = 2018
np.random.seed(seed)

current_model = DenseNet121

# Training setting - What kind of data to use
if setting in ['gasf','gadf', 'mtf', 'rp']:
    input_shape = (150,150,1)
elif setting=='stft':
    input_shape = (129, 120, 1)
else:
    raise ValueError('No valid data setting provided')

# More settings
model_name = model_name + '_' + setting + '_' + current_model.__name__
top_weights_path = TEMP_DATADIR + str(model_name) + '.h5'
logfile = TEMP_DATADIR + str(model_name) + '.log'
batch_size = 32
monitor = 'val_acc'
es_patience = 7
rlr_patience = 3
target_names = np.unique(y)

y = LabelEncoder().fit_transform(y)
X = X_names#np.array(X_names).reshape(-1,1)

X, y = shuffle(X, y, random_state=0) 

if undersampling:
    from imblearn.under_sampling import RandomUnderSampler
    ros = RandomUnderSampler(random_state=0)
    ros.fit(X,y)
    X, y = ros.fit_resample(X,y)
    X = pd.Series(X.ravel()).tolist()
    print('After undersampling: \n{}\n'.format(pd.DataFrame(y).iloc[:,0].value_counts()))
else:
    print('Class balance: \n{}\n'.format(pd.DataFrame(y).iloc[:,0].value_counts()))

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,y, random_state=0)

# Model parameters
img_input = Input(shape = input_shape)
model = current_model(input_tensor = img_input, classes = len(target_names), weights = None)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# Define Callbacks
callbacks_list = [ModelCheckpoint(monitor = monitor,
                                filepath = top_weights_path,
                                save_best_only = True,
                                save_weights_only = True,
                                verbose = 1),
                    EarlyStopping(monitor = monitor,
                                patience = es_patience,
                                verbose = 1),
                    ReduceLROnPlateau(monitor = monitor,
                                factor = 0.1,
                                patience = rlr_patience,
                                verbose = 1),
                    CSVLogger(filename = logfile)]
# TRAIN
model.fit_generator(train_generator(X_train,
                                    y_train, 
                                    batch_size=batch_size, 
                                    target_names=target_names,
                                    setting=setting),
                    steps_per_epoch = int(math.ceil(float(len(X_train)) / float(batch_size))),
                    epochs=100, 
                    validation_data = valid_generator(X_val,
                                                    y_val, 
                                                    batch_size=batch_size, 
                                                    target_names=target_names,
                                                    setting=setting), 
                    validation_steps = int(math.ceil(float(len(X_test)) / float(batch_size))),
                    callbacks = callbacks_list)
# EVALUATE
model.load_weights(top_weights_path)
loss, acc = model.evaluate_generator(valid_generator(X_test, 
                                                    y_test, 
                                                    batch_size=batch_size, 
                                                    setting=setting, 
                                                    target_names=target_names),
        steps = int(math.ceil(float(len(X_test)) / float(batch_size))))

print('loss', loss)
print('Test accuracy:', acc)

# SAVING
from keras.models import model_from_yaml
model_yaml = model.to_yaml()
with open(TEMP_DATADIR + model_name + ".yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
model.save_weights(TEMP_DATADIR + model_name + "_weights.h5")